# Aligning protein sequences with UniProt and EBI Clustal Omega using Python

In this notebook, I am showing how to easily align the desired protein sequences through UniProt (www.uniprot.org) and EBI (www.ebi.ac.uk) websites with Python.

Link to Colab:

In [ ]:
"https://colab.research.google.com/github/edu9as/web-scraping/blob/master/Aligning-Sequences-With-UniProt-And-Clustal-Omega.ipynb"

## Loading libraries

First of all, I am loading all the Python libraries that are needed to do this task:

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import random
import re

## Obtaining protein sequences

Before any web scraping, it is very important to read the ```robots.txt``` of the site you are crawling. What are the restrictions for crawlers in UniProt? Let's take a look it:

In [2]:
uniprot_robots = requests.get("https://www.uniprot.org/robots.txt").text
print(uniprot_robots)


User-agent: *
Disallow: /jobs/
Disallow: /uniref/UniRef
Disallow: /uniparc/UPI
Disallow: /uniprot/*.fasta
Disallow: /uniprot/*.xml
Disallow: /uniprot/*.txt
Disallow: /blast/
Disallow: /batch/
Disallow: /align/
Disallow: /mapping/
Disallow: /uploadlists/
Disallow: /peptidesearch/
Disallow: /contact
Disallow: /status
Disallow: /limbo
Allow: /uniprot/*.xml.gz
Disallow: /*?
Sitemap: https://www.uniprot.org/sitemap.xml



I was thinking of getting the sequences about some given proteins using UniProt, so most of these restrictions aren't important for this work. However, we cannot obtain the protein sequences from their fasta files because this is disallowed by UniProt (*Disallow: /uniprot/*.fasta*), so we should access them in another way.

In the next cell I am defining a function to obtain the sequence of the protein in the UniProt entry of interest. The only thing the user has to do is to introduce a protein name (i.e., the text he/she would write in the UniProt search bar) and select which of the hits corresponds with the desired protein. The output of this function is the protein sequence, which is retrieved avoiding **/uniprot/\*.fasta** webpages (which were disallowed, as we saw in the previous cell).

In [3]:
def uniprot_entry(name):
    up_query_url = "https://www.uniprot.org/uniprot/?query={}&sort=score"
    up_entry_url = "https://www.uniprot.org/uniprot/{}"
    query_url = up_query_url.format(name)
    seen_hits = 0
    while True:
            query_page = requests.get(query_url).text
            query_up = bs(query_page, "html.parser")

            up_hits = query_up.findAll("td", class_="entryID")
            up_total_hits = query_up.find("strong",
                                          class_="queryResultCount")
            print("There are {} hits for this protein name and you have "
                  "seen {}. Please, select UniProt entry:"
                  "".format(up_total_hits.text, seen_hits))
            while True:
                for hit in up_hits:
                    seen_hits += 1
                    row = hit.parent.findAll("td")
                    info = [cell for cell in row]
                    id_ = info[2].text
                    name = info[4].find("div", class_="long").text
                    sp = info[6].text
                    length = info[7].text
                    prompt = ("- Entry {}: {}.\n"
                              "-Species: {}\n-Length: {} aa.\n  ").format(id_, 
                                                                          name, 
                                                                          sp, 
                                                                          length)
                    correct_protein = input(prompt) != ""
                    if correct_protein:
                        hit_url = up_entry_url.format(hit.text)
                        uniprot_page = requests.get(hit_url).text
                        soup = bs(uniprot_page, "html.parser")
                        sequence = soup.find("pre", class_="sequence").text
                        for thing in " 0123456789":
                            sequence = sequence.replace(thing, "")
                        sequence = ">{} | {} | {}\n{}".format(hit.text, name, sp, sequence)
                        return sequence
                    elif seen_hits % 25 == 0:
                        if seen_hits == 25:
                            up_query_url = query_url+"&offset={}"
                        query_url = up_query_url.format(seen_hits)
                        break
                if correct_protein or seen_hits % 25 == 0: 
                    break

<!-- Hola -->

For example, let's obtain the sequence of human TRPA1. Why? Because TRPA1 is the best protein in the world. We query for "human trpa1", and we select the first entry.

In [4]:
sequence = uniprot_entry("human trpa1")

There are 19 hits for this protein name and you have seen 0. Please, select UniProt entry:
- Entry TRPA1_HUMAN: Transient receptor potential cation channel subfamily A member 1 (Ankyrin-like with transmembrane domains protein 1)  (Transformation-sensitive protein p120)  (Wasabi receptor) .
-Species: Homo sapiens (Human)
-Length: 1,119 aa.
  y


This the amino-acid sequence of human TRPA1 in fasta format. The header has been formatted as:

\> ```UniProt ID``` | ```Protein name (long)``` | ```Organism```. 

In [5]:
print(sequence)

>O75762 | Transient receptor potential cation channel subfamily A member 1 (Ankyrin-like with transmembrane domains protein 1)  (Transformation-sensitive protein p120)  (Wasabi receptor)  | Homo sapiens (Human)
MKRSLRKMWRPGEKKEPQGVVYEDVPDDTEDFKESLKVVFEGSAYGLQNFNKQKKLKRCDDMDTFFLHYAAAEGQIELMEKITRDSSLEVLHEMDDYGNTPLHCAVEKNQIESVKFLLSRGANPNLRNFNMMAPLHIAVQGMNNEVMKVLLEHRTIDVNLEGENGNTAVIIACTTNNSEALQILLKKGAKPCKSNKWGCFPIHQAAFSGSKECMEIILRFGEEHGYSRQLHINFMNNGKATPLHLAVQNGDLEMIKMCLDNGAQIDPVEKGRCTAIHFAATQGATEIVKLMISSYSGSVDIVNTTDGCHETMLHRASLFDHHELADYLISVGADINKIDSEGRSPLILATASASWNIVNLLLSKGAQVDIKDNFGRNFLHLTVQQPYGLKNLRPEFMQMQQIKELVMDEDNDGCTPLHYACRQGGPGSVNNLLGFNVSIHSKSKDKKSPLHFAASYGRINTCQRLLQDISDTRLLNEGDLHGMTPLHLAAKNGHDKVVQLLLKKGALFLSDHNGWTALHHASMGGYTQTMKVILDTNLKCTDRLDEDGNTALHFAAREGHAKAVALLLSHNADIVLNKQQASFLHLALHNKRKEVVLTIIRSKRWDECLKIFSHNSPGNKCPITEMIEYLPECMKVLLDFCMLHSTEDKSCRDYYIEYNFKYLQCPLEFTKKTPTQDVIYEPLTALNAMVQNNRIELLNHPVCKEYLLMKWLAYGFRAHMMNLGSYCLGLIPMTILVVNIKPGMAFNSTGIINETSDHSEILDTTNSYLIKTCMILVFLSSIFGYCKEA

We can also tweak a little bit the function we define earlier, so as to obtain a list with the sequences of all the hits (indeed, the 25 hits with the highest score) for a given query:

In [6]:
def uniprot_entries(name):
    up_query_url = "https://www.uniprot.org/uniprot/?query={}&sort=score"
    up_entry_url = "https://www.uniprot.org/uniprot/{}"
    query_url = up_query_url.format(name)
    seen_hits = 0
    while True:
            query_page = requests.get(query_url).text
            query_up = bs(query_page, "html.parser")

            up_hits = query_up.findAll("td", class_="entryID")
            
            hits_list = []
            
            for hit in up_hits:
                row = hit.parent.findAll("td")
                info = [cell for cell in row]
                id_ = info[2].text
                name = info[4].find("div", class_="long").text
                sp = info[6].text
                length = info[7].text
                hit_url = up_entry_url.format(hit.text)
                uniprot_page = requests.get(hit_url).text
                soup = bs(uniprot_page, "html.parser")
                sequence = soup.find("pre", class_="sequence").text
                for thing in " 0123456789":
                    sequence = sequence.replace(thing, "")
                sequence = ">{} | {} | {}\n{}".format(hit.text, name, sp, sequence)
                hits_list.append((hit.text, name, sp, sequence))
                print(hit.text, end = ", ")
                time.sleep(3)
            return hits_list

We obtain the sequences of the 25 UniProt entries with best scores when we search for "trpa1":

In [7]:
trpa1_homologs = uniprot_entries("trpa1")

O75762, Q8BLA8, Q6RI86, Q7Z020, O22765, Q18297, G8IQ59, H0USS9, M9PBX8, B6D5I5, Q5UM14, F1R8I6, Q5UM15, A0A2I2UZQ7, F1RWI1, G5E522, B6D5I6, A0A1D5QMT1, F1R7I3, P83480, Q9CQG9, F1LRH9, Q9NV29, C0HLG4, Q00560, 

This new object, ```trpa1_homologs```, consists of a list of tuples. There is a tuple with 4 positions for each entry: UniProt ID, Name of the protein (long), Organism where the protein is expressed, and the sequence in fasta format.

In [8]:
trpa1_homologs

[('O75762',
  'Transient receptor potential cation channel subfamily A member 1 (Ankyrin-like with transmembrane domains protein 1)  (Transformation-sensitive protein p120)  (Wasabi receptor) ',
  'Homo sapiens (Human)',
  '>O75762 | Transient receptor potential cation channel subfamily A member 1 (Ankyrin-like with transmembrane domains protein 1)  (Transformation-sensitive protein p120)  (Wasabi receptor)  | Homo sapiens (Human)\nMKRSLRKMWRPGEKKEPQGVVYEDVPDDTEDFKESLKVVFEGSAYGLQNFNKQKKLKRCDDMDTFFLHYAAAEGQIELMEKITRDSSLEVLHEMDDYGNTPLHCAVEKNQIESVKFLLSRGANPNLRNFNMMAPLHIAVQGMNNEVMKVLLEHRTIDVNLEGENGNTAVIIACTTNNSEALQILLKKGAKPCKSNKWGCFPIHQAAFSGSKECMEIILRFGEEHGYSRQLHINFMNNGKATPLHLAVQNGDLEMIKMCLDNGAQIDPVEKGRCTAIHFAATQGATEIVKLMISSYSGSVDIVNTTDGCHETMLHRASLFDHHELADYLISVGADINKIDSEGRSPLILATASASWNIVNLLLSKGAQVDIKDNFGRNFLHLTVQQPYGLKNLRPEFMQMQQIKELVMDEDNDGCTPLHYACRQGGPGSVNNLLGFNVSIHSKSKDKKSPLHFAASYGRINTCQRLLQDISDTRLLNEGDLHGMTPLHLAAKNGHDKVVQLLLKKGALFLSDHNGWTALHHASMGGYTQTMKVILDTNLKCTDRLDEDGNTALHFAAREGHAKAV

## Alignment of sequences using Clustal Omega

The next step is to join all the sequences in the previous list into a single string. The sequences will be separated by two newline symbols:

In [9]:
sequences = "\n\n".join([thing[3] for thing in trpa1_homologs])
print(sequences)

>O75762 | Transient receptor potential cation channel subfamily A member 1 (Ankyrin-like with transmembrane domains protein 1)  (Transformation-sensitive protein p120)  (Wasabi receptor)  | Homo sapiens (Human)
MKRSLRKMWRPGEKKEPQGVVYEDVPDDTEDFKESLKVVFEGSAYGLQNFNKQKKLKRCDDMDTFFLHYAAAEGQIELMEKITRDSSLEVLHEMDDYGNTPLHCAVEKNQIESVKFLLSRGANPNLRNFNMMAPLHIAVQGMNNEVMKVLLEHRTIDVNLEGENGNTAVIIACTTNNSEALQILLKKGAKPCKSNKWGCFPIHQAAFSGSKECMEIILRFGEEHGYSRQLHINFMNNGKATPLHLAVQNGDLEMIKMCLDNGAQIDPVEKGRCTAIHFAATQGATEIVKLMISSYSGSVDIVNTTDGCHETMLHRASLFDHHELADYLISVGADINKIDSEGRSPLILATASASWNIVNLLLSKGAQVDIKDNFGRNFLHLTVQQPYGLKNLRPEFMQMQQIKELVMDEDNDGCTPLHYACRQGGPGSVNNLLGFNVSIHSKSKDKKSPLHFAASYGRINTCQRLLQDISDTRLLNEGDLHGMTPLHLAAKNGHDKVVQLLLKKGALFLSDHNGWTALHHASMGGYTQTMKVILDTNLKCTDRLDEDGNTALHFAAREGHAKAVALLLSHNADIVLNKQQASFLHLALHNKRKEVVLTIIRSKRWDECLKIFSHNSPGNKCPITEMIEYLPECMKVLLDFCMLHSTEDKSCRDYYIEYNFKYLQCPLEFTKKTPTQDVIYEPLTALNAMVQNNRIELLNHPVCKEYLLMKWLAYGFRAHMMNLGSYCLGLIPMTILVVNIKPGMAFNSTGIINETSDHSEILDTTNSYLIKTCMILVFLSSIFGYCKEA

Before using EBI Clustal Omega webpage, let's analyze its ```robots.txt``` file:

In [10]:
print(requests.get("https://www.ebi.ac.uk/robots.txt").text)

# Robots exclusion file for http://www.ebi.ac.uk/
# See https://gitlab.ebi.ac.uk/ebiwd/ebi.ac.uk-root-assets
# Any questions/comments to www-dev@ebi.ac.uk

User-agent: sogou spider
Disallow: /Tools/services

User-agent: Baiduspider
Disallow: /chebi

User-agent: MJ12bot
Disallow: /

User-agent: naverbot
Disallow: /

User-agent: SemrushBot
Disallow: /biomodels

User-agent: SemrushBot-SA
Disallow: /biomodels

User-agent: Twiceler
Disallow: /

User-agent: discobot 
Disallow: /

#User-agent: msnbot
#Disallow: /

User-agent: EBISearch
Crawl-Delay: 0

User-agent: *
Crawl-Delay: 10
Disallow: /*private*
Disallow: /~
Disallow: /3D
Disallow: /abc
Disallow: /awstats
Disallow: /beta/
Disallow: /bioinformatics
Disallow: /biosamples/beta
Disallow: /biomodels/papers
Disallow: /cgi-bin/printable
Disallow: /cgi-bin/webservices
Disallow: /cgi-bin/*?
Disallow: /cgi_files
Disallow: /citexplore
Disallow: /Compass
Disallow: /contrib
Disallow: /cpg
Disallow: /Databases/Genome_MOT/
Disallow: /dbases
Disallow: 

The alignment we are performing here only uses the */Tools/* group of webpages. So, everything is okay.

Now, we can define a function that posts information to the EBI webpage (i.e., the sequences and the email account where the results will be sent) and then gets the results from the appropriate webpage. As a job title, a string is generated randomly: ```job_RandomNumberBetween0and1000000```.

In [11]:
def clustal_align(sequences, email):
    """
    Performs an alignment of the required protein sequences using EBI 
    Clustal Omega website. The result is printed to the console and sent to
    the email introduced. User can introduce a list of Protein instances,
    or the user will be prompted to enter the name of the proteins to be
    aligned and indicate the desired UniProt entry.        

    Parameters
    ----------
    sequences : list, optional
        List of Protein instances to be aligned. If False, the user is
        prompted to enter the names of the proteins whose sequences are to
        be aligned. The default is False.
    email : string
        Email where the result of the alignment will be sent. Besides, this
        result will be printed to the console.

    Returns
    -------
    None.

    """
    clustal = "https://www.ebi.ac.uk/Tools/services/web_clustalo/toolform.ebi"

    myobj = {'tool': 'clustalo',
             'stype': "protein",
             'sequence': sequences,
             'outfmt': "clustal_num",
             'notification': 'on',
             'email': email,
             'title': "job_" + str(random.randint(0,1e6)),
             'submit': 'Submit'}

    session = requests.session()
    x = requests.post(clustal, data = myobj, allow_redirects=True)
    i = 0
    while True:
        try:
            print("\b", end = "")
            suffix = bs(x.text, "html.parser").find("div", id="ebi_mainContent").a.get("href")
            result_url = "https://www.ebi.ac.uk/Tools/services/web_clustalo/" + suffix

            page = requests.get(result_url).text
            soup = bs(page, "html.parser")
            
            result = soup.find("pre").text

            print(result)
            return result
        except:
            print("\bAligning sequences...", end = "")
            time.sleep((sequences.count("\n") + 2) / 3)
            if i > 3:
                return None
            i += 1

Let's align the TRPA1 homologs with this function. I am hiding here my email account as a matter of privacy:

In [12]:
aligned_sequences = clustal_align(sequences, email="*****")

Aligning sequences.Aligning sequences.Aligning sequences..CLUSTAL O(1.2.4) multiple sequence alignment


O75762          ------------------------------------------------------------	0
Q8BLA8          ------------------------------------------------------------	0
Q6RI86          ------------------------------------------------------------	0
Q7Z020          ---------------------MTSGDKETPKRE---------------------DFASAL	18
O22765          -------------------------MDLLKTPS---------------------------	8
Q18297          ------------------------------------------------------------	0
G8IQ59          MPKLYNGVYSGQCGALSPPDLMEAQPKLLPKPRSNSSGSTGRNSKYWIFSMIIERSAGPK	60
H0USS9          MPKLYNGVYSGQCGALSPPDLMEAQPKLLPKPRSNSSGSTGRNSKYWIFSMIIERSAGPK	60
M9PBX8          MPKLYNGVYSGQCGALSPPDLMEAQPKLLPKPRSNSSGSTGRNSKYWIFSMIIERSAGPK	60
B6D5I5          ------------------------------------------------------------	0
Q5UM14          ------------------------------------------------------------	0
F1R8I6          ------

Finally, we can transform this output to a Pandas dataframe. This dataframe might be employed for future experiments, such as clustering the sequences into clusters depending on their similarities making use of machine learning.

In [ ]:
alignment_dict = {}
for i in range(29):
    aligned_chunk = [line for line in aligned_sequences.split("\n")[3:] if line.count("\t") > 0][i*25 : i*25+25]
    for line in aligned_chunk:
        id_ = re.search("(\w+)\W+(\w+)\W+(\w+)", line.replace("-", "_")).group(1)
        seq = list(re.search("(\w+)\W+(\w+)\W+(\w+)", line.replace("-", "_")).group(2))
        if i == 0:
            alignment_dict[id_] = []
        alignment_dict[id_] += seq

And the database is created:

In [14]:
a = pd.DataFrame(alignment_dict)
a.index = ["P" + str(i) for i in range(1690)]
a = a.T
a

,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P1680,P1681,P1682,P1683,P1684,P1685,P1686,P1687,P1688,P1689
O75762,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_
Q8BLA8,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_
Q6RI86,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_
Q7Z020,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_
O22765,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_
Q18297,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_
G8IQ59,M,P,K,L,Y,N,G,V,Y,S,...,_,_,_,_,_,_,_,_,_,_
H0USS9,M,P,K,L,Y,N,G,V,Y,S,...,_,_,_,_,_,_,_,_,_,_
M9PBX8,M,P,K,L,Y,N,G,V,Y,S,...,_,_,_,_,_,_,_,_,_,_
B6D5I5,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_


## End of the notebook

In this notebook, I have combined web scraping techniques with some biology knowledge to create a dataset with the result of an alignment of 25 sequences retrieved from UniProt making use of EBI Clustal Omega functionality.